# P.1 Catch the phase
Learning outcomes

* State the motivation behind the quantum phase estimation (QPE) subroutine.
* List some applications of QPE.
* Describe the phase kickback technique.
* Estimate a phase using phase kickback.

Suppose that we apply the controlled unitary $U$ to its eigenvector $\Ket{\psi}$, so that
$$U\Ket{\psi} = e^{2\pi i\theta}\Ket{\psi}$$
Then, the control wires pick up a phase of $e^{2\pi i\theta}$ resulting in the global phase getting encoded into a relative phase. The phase kickback circuit is shown below:
![circuit](./images/P.1.1.png)
To illustrate this concept, we will implement this circuit to see if we can detect a phase of -1.

In [1]:
import pennylane as qml
import pennylane.numpy as np

Calculation

In [5]:
import sympy as sp

In [21]:
dev = qml.device("default.qubit", wires=2, shots = 1)

@qml.qnode(dev)
def guess_the_unitary(unitary):
    """Given a unitary that performs a Z or a -Z operation
    on a qubit, guess which one it is.

    Args:
        U (array[complex]): A unitary matrix, guaranteed to be either Z or -Z.

    Returns:
        array [int]:  Probabilities on  on the first qubit
        using qml.probs()
    """
    qml.Hadamard(wires=0)
    qml.ControlledQubitUnitary(U,control_wires=0,wires=1)
    qml.Hadamard(wires=0)
    #qml.PauliX(wires=0)
    return qml.sample(wires=0)

In [26]:
# Z gate
U = qml.PauliZ.compute_matrix()

# -Z gate
#U = (-1)*qml.PauliZ.compute_matrix()
U

array([[ 1,  0],
       [ 0, -1]])

In [27]:
print(qml.draw(guess_the_unitary)(U),'\n')
print(guess_the_unitary(U))

0: ──H─╭●──────H─┤  Sample
1: ────╰U(M0)────┤        

M0 = 
[[ 1  0]
 [ 0 -1]] 

0


**Codercise P.1.1.**
You are given a unitary that is promised to be either the Z gate or the -Z gate. Write a quantum program using phase kickback that will result in the state  with a probability of 100% on the first qubit if Z is applied and  with a probability of 100% on the first qubit if -Z is applied.
Hint
Use qml.ControlledQubitUnitary or qml.ctrl to apply unitaries with control bits.

In [4]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def guess_the_unitary(unitary):
    """Given a unitary that performs a Z or a -Z operation
    on a qubit, guess which one it is.

    Args:
        U (array[complex]): A unitary matrix, guaranteed to be either Z or -Z.

    Returns:
        array [int]:  Probabilities on  on the first qubit
        using qml.probs()
    """
    qml.Hadamard(wires=0)
    qml.ControlledQubitUnitary(unitary,control_wires=0,wires=1)
    qml.Hadamard(wires=0)

    return qml.probs(wires=0)

# Z gate
#U = qml.PauliZ.compute_matrix()

# -Z gate
U = (-1)*qml.PauliZ.compute_matrix()

print(guess_the_unitary(U))

[1. 0.]


**Codercise P.1.2.**
Find the eigenvalues of the X gate by following the steps outlined below:
1. Prepare an eigenvector of X as input to the target wire.
2. Apply the phase kickback circuit.
3. Obtain the probabilities on the control wire.
4. Use the probabilities $P(0)$ and $P(1)$ to calculate the eigenvalues.
5. Modify the variable `done` and `True`.

Note that the eigenvectors of the X gate are $\Ket{+}$ and $\Ket{-}$ and are supplied to the function `phase_kickback_X` as strings "plus" and "minus".

In [31]:
dev2 = qml.device("default.qubit", wires=2)

@qml.qnode(dev2)
def phase_kickback_X(eigenvector):
    """ Given an eigenvector of X,
    apply the phase kickback circuit to observe
    the probabilities on the control wire

    Args:
        eigenvector(String): A string "plus" or "minus" depicting
        the eigenvector of X

    Returns:
        array[int]: Measurement outcome on the first qubit using qml.probs()
    """
    # Prepare |ψ>
    if eigenvector != "plus":
        qml.PauliX(wires = 1)

    qml.Hadamard(wires = 1)


    # Phase kickback
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)

    return qml.probs(wires=[0])

print(phase_kickback_X("plus"))
print(phase_kickback_X("minus"))

# MODIFY EIGENVALUES BELOW
eigenvalue_of_X_plus = 1
eigenvalue_of_X_minus = -1

[1. 0.]
[0. 1.]
